<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 561 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 17.7 MB/s 


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    51
1    10
Name: InPortfolio, dtype: int64

In [6]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1060.0

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [8]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [9]:
stock_prec_dev('^NSEI').tail(5)

[*********************100%***********************]  1 of 1 completed


,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,,
2022-12-09,18497.0,56.0,^NSEI,6.25,0.74,0.60,18812.0,15294.0,91.03,8.97
2022-12-12,18497.0,56.0,^NSEI,6.18,0.73,0.54,18812.0,15294.0,91.05,8.95
2022-12-13,18608.0,60.0,^NSEI,6.75,0.73,0.51,18812.0,15294.0,94.20,5.80
2022-12-14,18660.0,62.0,^NSEI,6.97,0.71,0.51,18812.0,15294.0,95.69,4.31
2022-12-15,18415.0,50.0,^NSEI,5.51,0.68,0.45,18812.0,15294.0,88.71,11.29


In [10]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [11]:
# heavy upside
df_prec_dev.sort_values(by = 'Upside%', ascending=False).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-15,453.0,35.0,ICICIPRULI.NS,-12.88,-1.15,-1.43,616.0,447.0,3.55,96.45,640.0,41.28
1,2022-12-15,460.0,56.0,GODREJIND.NS,-1.82,0.00,1.35,644.0,405.0,23.20,76.80,638.0,38.70
2,2022-12-15,708.0,68.0,MOTILALOFS.NS,-7.88,-1.43,-0.58,980.0,654.0,16.69,83.31,930.0,31.36
3,2022-12-15,1274.0,54.0,SBILIFE.NS,4.90,0.49,0.16,1332.0,1025.0,80.94,19.06,1604.0,25.90
4,2022-12-15,524.0,35.0,JUBLFOOD.NS,-10.91,-0.85,-1.42,786.0,464.0,18.59,81.41,649.0,23.85
5,2022-12-15,6625.0,40.0,BAJFINANCE.NS,-3.00,-0.34,-0.92,7853.0,5284.0,52.21,47.79,8015.0,20.98
6,2022-12-15,909.0,42.0,ICICIBANK.NS,8.45,1.07,0.33,953.0,654.0,85.22,14.78,1094.0,20.35
7,2022-12-15,417.0,44.0,TATAMOTORS.NS,-4.65,-0.23,-0.47,525.0,372.0,29.22,70.78,495.0,18.71
8,2022-12-15,2523.0,35.0,TITAN.NS,1.44,0.44,-0.31,2770.0,1936.0,70.35,29.65,2989.0,18.47
9,2022-12-15,258.0,45.0,ZEEL.NS,-3.26,0.00,0.00,368.0,209.0,31.01,68.99,303.0,17.44


In [12]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-15,453.0,35.0,ICICIPRULI.NS,-12.88,-1.15,-1.43,616.0,447.0,3.55,96.45,640.0,41.28
34,2022-12-15,395.0,45.0,WIPRO.NS,-12.57,-1.11,-0.25,722.0,375.0,5.82,94.18,425.0,7.59
16,2022-12-15,1023.0,40.0,TECHM.NS,-11.98,-0.86,-0.38,1806.0,965.0,6.87,93.13,1158.0,13.20
4,2022-12-15,524.0,35.0,JUBLFOOD.NS,-10.91,-0.85,-1.42,786.0,464.0,18.59,81.41,649.0,23.85
42,2022-12-15,3365.0,48.0,DIVISLAB.NS,-10.66,-1.17,-0.61,4678.0,3266.0,7.04,92.96,3560.0,5.79
47,2022-12-14,4443.0,37.0,LTI.NS,-10.47,-0.73,-1.44,7562.0,3824.0,16.56,83.44,4615.0,3.87
2,2022-12-15,708.0,68.0,MOTILALOFS.NS,-7.88,-1.43,-0.58,980.0,654.0,16.69,83.31,930.0,31.36
14,2022-12-15,1155.0,31.0,HAVELLS.NS,-7.78,-0.48,-1.22,1406.0,1070.0,25.19,74.81,1328.0,14.98
17,2022-12-15,84.0,51.0,IBREALEST.NS,-7.58,-1.10,1.22,174.0,60.0,21.14,78.86,95.0,13.10
12,2022-12-15,78.0,55.0,NATIONALUM.NS,-7.50,0.00,1.32,130.0,67.0,16.98,83.02,90.0,15.38


In [13]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
55,2022-12-15,320.0,57.0,CANBK.NS,28.20,2.80,1.69,329.0,175.0,94.48,5.52,310.0,-3.12
32,2022-12-15,138.0,59.0,FEDERALBNK.NS,20.92,2.63,0.76,138.0,81.0,99.74,0.26,149.0,7.97
18,2022-12-15,938.0,71.0,AXISBANK.NS,19.97,1.92,1.84,945.0,626.0,97.85,2.15,1060.0,13.01
59,2022-12-15,4465.0,75.0,BRITANNIA.NS,18.16,1.80,1.65,4465.0,3094.0,100.03,-0.03,3935.0,-11.87
20,2022-12-15,1240.0,64.0,INDUSINDBK.NS,16.94,1.42,1.11,1264.0,778.0,94.98,5.02,1393.0,12.34
58,2022-12-15,2178.0,71.0,LT.NS,15.70,1.43,1.58,2195.0,1472.0,97.59,2.41,2036.0,-6.52
13,2022-12-15,616.0,58.0,SBIN.NS,14.70,1.49,1.19,626.0,435.0,94.74,5.26,710.0,15.26
15,2022-12-15,1287.0,55.0,M&M.NS,12.69,1.14,0.16,1354.0,691.0,89.89,10.11,1468.0,14.06
48,2022-12-15,339.0,45.0,ITC.NS,11.46,1.32,0.00,361.0,208.0,85.52,14.48,350.0,3.24
22,2022-12-15,1632.0,60.0,HDFCBANK.NS,10.11,1.01,1.15,1662.0,1281.0,92.07,7.93,1813.0,11.09


In [14]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-15,453.0,35.0,ICICIPRULI.NS,-12.88,-1.15,-1.43,616.0,447.0,3.55,96.45,640.0,41.28
2,2022-12-15,708.0,68.0,MOTILALOFS.NS,-7.88,-1.43,-0.58,980.0,654.0,16.69,83.31,930.0,31.36
3,2022-12-15,1274.0,54.0,SBILIFE.NS,4.90,0.49,0.16,1332.0,1025.0,80.94,19.06,1604.0,25.90
4,2022-12-15,524.0,35.0,JUBLFOOD.NS,-10.91,-0.85,-1.42,786.0,464.0,18.59,81.41,649.0,23.85
10,2022-12-15,580.0,60.0,HDFCLIFE.NS,2.15,0.35,0.71,674.0,504.0,44.82,55.18,681.0,17.41
16,2022-12-15,1023.0,40.0,TECHM.NS,-11.98,-0.86,-0.38,1806.0,965.0,6.87,93.13,1158.0,13.20
19,2022-12-15,4140.0,59.0,NAUKRI.NS,-1.74,-0.36,0.62,5696.0,3375.0,32.95,67.05,4677.0,12.97
21,2022-12-15,1540.0,42.0,INFY.NS,-1.75,0.13,0.00,1940.0,1365.0,30.52,69.48,1715.0,11.36
24,2022-12-15,1102.0,53.0,MUTHOOTFIN.NS,-3.31,-0.26,0.46,1540.0,956.0,25.04,74.96,1217.0,10.44
42,2022-12-15,3365.0,48.0,DIVISLAB.NS,-10.66,-1.17,-0.61,4678.0,3266.0,7.04,92.96,3560.0,5.79


In [15]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     21.326720
2     16.201695
3     13.380864
4     12.321761
10     8.994627
16     6.819591
19     6.700765
21     5.868981
24     5.393676
42     2.991321
Name: Upside%, dtype: float64

In [16]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
1,2022-12-15,460.0,56.0,GODREJIND.NS,-1.82,0.00,1.35,644.0,405.0,23.20,76.80,638.0,38.70
5,2022-12-15,6625.0,40.0,BAJFINANCE.NS,-3.00,-0.34,-0.92,7853.0,5284.0,52.21,47.79,8015.0,20.98
6,2022-12-15,909.0,42.0,ICICIBANK.NS,8.45,1.07,0.33,953.0,654.0,85.22,14.78,1094.0,20.35
7,2022-12-15,417.0,44.0,TATAMOTORS.NS,-4.65,-0.23,-0.47,525.0,372.0,29.22,70.78,495.0,18.71
8,2022-12-15,2523.0,35.0,TITAN.NS,1.44,0.44,-0.31,2770.0,1936.0,70.35,29.65,2989.0,18.47
9,2022-12-15,258.0,45.0,ZEEL.NS,-3.26,0.00,0.00,368.0,209.0,31.01,68.99,303.0,17.44
11,2022-12-15,3304.0,40.0,EICHERMOT.NS,4.74,0.54,-0.70,3850.0,2256.0,65.72,34.28,3865.0,16.98
12,2022-12-15,78.0,55.0,NATIONALUM.NS,-7.50,0.00,1.32,130.0,67.0,16.98,83.02,90.0,15.38
13,2022-12-15,616.0,58.0,SBIN.NS,14.70,1.49,1.19,626.0,435.0,94.74,5.26,710.0,15.26
14,2022-12-15,1155.0,31.0,HAVELLS.NS,-7.78,-0.48,-1.22,1406.0,1070.0,25.19,74.81,1328.0,14.98


In [17]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-15,524.0,35.0,JUBLFOOD.NS,-10.91,-0.85,-1.42,786.0,464.0,18.59,81.41,649.0,23.85
8,2022-12-15,2523.0,35.0,TITAN.NS,1.44,0.44,-0.31,2770.0,1936.0,70.35,29.65,2989.0,18.47
14,2022-12-15,1155.0,31.0,HAVELLS.NS,-7.78,-0.48,-1.22,1406.0,1070.0,25.19,74.81,1328.0,14.98
16,2022-12-15,1023.0,40.0,TECHM.NS,-11.98,-0.86,-0.38,1806.0,965.0,6.87,93.13,1158.0,13.20
21,2022-12-15,1540.0,42.0,INFY.NS,-1.75,0.13,0.00,1940.0,1365.0,30.52,69.48,1715.0,11.36
33,2022-12-15,605.0,44.0,BERGEPAINT.NS,-5.84,-0.47,-0.16,791.0,561.0,19.33,80.67,651.0,7.60
34,2022-12-15,395.0,45.0,WIPRO.NS,-12.57,-1.11,-0.25,722.0,375.0,5.82,94.18,425.0,7.59
35,2022-12-15,3131.0,46.0,ASIANPAINT.NS,-1.40,0.00,-0.03,3576.0,2580.0,55.28,44.72,3361.0,7.35
36,2022-12-15,1042.0,40.0,HCLTECH.NS,0.13,0.38,-0.38,1352.0,883.0,33.98,66.02,1117.0,7.20
39,2022-12-15,2663.0,56.0,HINDUNILVR.NS,7.52,0.85,0.85,2722.0,1944.0,92.45,7.55,2832.0,6.35


In [18]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
51,2022-12-15,3305.0,47.0,TCS.NS,-0.63,0.09,0.21,4019.0,2982.0,31.16,68.84,3385.0,2.42
21,2022-12-15,1540.0,42.0,INFY.NS,-1.75,0.13,0.00,1940.0,1365.0,30.52,69.48,1715.0,11.36
50,2022-11-22,3434.0,47.0,MINDTREE.NS,-2.28,0.14,0.62,4842.0,2742.0,32.95,67.05,3520.0,2.50
27,2022-12-15,8584.0,34.0,MARUTI.NS,0.71,0.19,-0.62,9528.0,6769.0,65.79,34.21,9356.0,8.99
30,2022-12-15,4469.0,54.0,DRREDDY.NS,2.66,0.21,0.25,4907.0,3719.0,63.13,36.87,4829.0,8.06
25,2022-12-15,1863.0,37.0,KOTAKBANK.NS,0.44,0.27,-0.26,1966.0,1642.0,68.27,31.73,2055.0,10.31
49,2022-12-15,1559.0,40.0,COLPAL.NS,-0.32,0.32,0.19,1678.0,1394.0,58.08,41.92,1606.0,3.01
10,2022-12-15,580.0,60.0,HDFCLIFE.NS,2.15,0.35,0.71,674.0,504.0,44.82,55.18,681.0,17.41
36,2022-12-15,1042.0,40.0,HCLTECH.NS,0.13,0.38,-0.38,1352.0,883.0,33.98,66.02,1117.0,7.20
29,2022-12-15,805.0,56.0,TATACONSUM.NS,3.86,0.39,0.51,849.0,660.0,76.67,23.33,871.0,8.20


In [19]:
# turning around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > 0)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
17,2022-12-15,84.0,51.0,IBREALEST.NS,-7.58,-1.10,1.22,174.0,60.0,21.14,78.86,95.0,13.10
19,2022-12-15,4140.0,59.0,NAUKRI.NS,-1.74,-0.36,0.62,5696.0,3375.0,32.95,67.05,4677.0,12.97
24,2022-12-15,1102.0,53.0,MUTHOOTFIN.NS,-3.31,-0.26,0.46,1540.0,956.0,25.04,74.96,1217.0,10.44


In [20]:
# bull run
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_200', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
58,2022-12-15,2178.0,71.0,LT.NS,15.70,1.43,1.58,2195.0,1472.0,97.59,2.41,2036.0,-6.52
22,2022-12-15,1632.0,60.0,HDFCBANK.NS,10.11,1.01,1.15,1662.0,1281.0,92.07,7.93,1813.0,11.09
26,2022-12-15,2672.0,56.0,HDFC.NS,8.85,0.98,1.05,2756.0,2053.0,88.10,11.90,2940.0,10.03
56,2022-12-15,939.0,66.0,MCDOWELL-N.NS,9.44,0.93,1.01,947.0,732.0,96.28,3.72,885.0,-5.75
46,2022-12-15,7114.0,57.0,ULTRACEMCO.NS,7.46,0.88,1.01,7870.0,5177.0,71.92,28.08,7420.0,4.30
37,2022-12-15,587.0,57.0,DABUR.NS,5.37,0.72,0.88,604.0,485.0,85.63,14.37,625.0,6.47
31,2022-12-15,2015.0,66.0,INDIGO.NS,7.87,0.64,1.59,2265.0,1549.0,65.08,34.92,2176.0,7.99
41,2022-12-15,903.0,61.0,GODREJCP.NS,5.64,0.47,0.69,968.0,665.0,78.61,21.39,958.0,6.09
29,2022-12-15,805.0,56.0,TATACONSUM.NS,3.86,0.39,0.51,849.0,660.0,76.67,23.33,871.0,8.20
10,2022-12-15,580.0,60.0,HDFCLIFE.NS,2.15,0.35,0.71,674.0,504.0,44.82,55.18,681.0,17.41


In [21]:
# about to turn around
df_prec_dev[(df_prec_dev['Slope%_200'] < 0) & (df_prec_dev['Slope%_50'] > df_prec_dev['Slope%_200'])].sort_values(by = 'Slope%_200', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
52,2022-12-15,4182.0,60.0,DMART.NS,0.36,-0.22,-0.02,4749.0,3231.0,62.65,37.35,4282.0,2.39
24,2022-12-15,1102.0,53.0,MUTHOOTFIN.NS,-3.31,-0.26,0.46,1540.0,956.0,25.04,74.96,1217.0,10.44
19,2022-12-15,4140.0,59.0,NAUKRI.NS,-1.74,-0.36,0.62,5696.0,3375.0,32.95,67.05,4677.0,12.97
33,2022-12-15,605.0,44.0,BERGEPAINT.NS,-5.84,-0.47,-0.16,791.0,561.0,19.33,80.67,651.0,7.60
16,2022-12-15,1023.0,40.0,TECHM.NS,-11.98,-0.86,-0.38,1806.0,965.0,6.87,93.13,1158.0,13.20
17,2022-12-15,84.0,51.0,IBREALEST.NS,-7.58,-1.10,1.22,174.0,60.0,21.14,78.86,95.0,13.10
34,2022-12-15,395.0,45.0,WIPRO.NS,-12.57,-1.11,-0.25,722.0,375.0,5.82,94.18,425.0,7.59
42,2022-12-15,3365.0,48.0,DIVISLAB.NS,-10.66,-1.17,-0.61,4678.0,3266.0,7.04,92.96,3560.0,5.79
2,2022-12-15,708.0,68.0,MOTILALOFS.NS,-7.88,-1.43,-0.58,980.0,654.0,16.69,83.31,930.0,31.36
